In [1]:
import numpy as np
import pandas as pd
import numpy_financial as npf
import time
import sys
sys.path.append('../')

from utils.data_generation.hedge_item_gen import credit_fv_delta,CreditGenerator
from utils.data_generation.hedge_instrument_gen import SwapGenerator
from utils.solvers.mip_drop_cplex import solve_with_cplex
from utils.solvers.mip_drop_cbc import solve_with_cbc
from utils.validation_functions.validation_cplex import validate_solution_cplex
from utils.validation_functions.validation_cbc import validate_solution_cbc

pd.set_option('display.float_format', lambda x: '{:,.3f}'.format(x))


## Fair Value Change Example 

In [2]:
principal = 40000
maturity = 4
credit_spread = 0.015
interest_rates = (0.07, 0.075)

rate0 = credit_spread + interest_rates[0]
rate1 = credit_spread + interest_rates[1]

# Monthly payment based on rate0 
r_month = rate0 / 12
payment = -npf.pmt(r_month, maturity, principal)

# Remaining payments at t=1
fv0 = -npf.pv(rate0, maturity-1, payment)
fv1 = -npf.pv(rate1, maturity-1, payment)

result = round(float(fv1 - fv0), 2)

# Print all variables with names
print("principal =", principal)
print("maturity =", maturity)
print("credit_spread =", credit_spread)
print("interest_rates =", interest_rates)
print("rate0 =", rate0)
print("rate1 =", rate1)
print(f"r_month = {r_month:.3f}")
print(f"payment = {payment:.3f}")
print(f"fv0     = {fv0:.3f}")
print(f"fv1     = {fv1:.3f}")
print(f"result  = {result:.3f}")


principal = 40000
maturity = 4
credit_spread = 0.015
interest_rates = (0.07, 0.075)
rate0 = 0.085
rate1 = 0.09
r_month = 0.007
payment = 10177.708
fv0     = 25994.095
fv1     = 25762.779
result  = -231.320


## Scenario Valdiation for FV change of Credits

### Increase of the Interest Rates

In [3]:
delta = credit_fv_delta(
    principal=40_000,
    maturity=4,
    credit_spread=0.015,             
    interest_rates=(0.07, 0.075)    
)
delta


-231.32

### Decrease of the Interest Rates

In [4]:
delta = credit_fv_delta(
    principal=40_000,
    maturity=4,
    credit_spread=0.015,             
    interest_rates=(0.075, 0.07)    
)
delta


231.55

## Credit Portfolio Generation

In [5]:
# New setting for updated CreditGenerator

num_credits = 10000
num_swaps = 2
fulfillment_ratio = 0.6


credit_generator_inputs = {
    'credit_types': ['Cash Loan Credit', 'Car Loan Credit', 'Mortgage Credit'],
    'principals_ranges': [[10_000, 50_000], [20_000, 70_000], [30_000, 100_000]],
    'maturities_ranges': [[4, 8], [3, 9], [2, 12]],
    'distributions': [0.90, 0.08, 0.02],  # 90% cash, 8% car, 2% mortgage
    'credit_spread_ranges': [
        [0.005, 0.015],   # Cash loans: tighter spreads (0.5% – 1.5%)
        [0.015, 0.030],   # Car loans: higher spreads (1.5% – 3.0%)
        [0.020, 0.040]    # Mortgage loans: even higher (2.0% – 4.0%)
    ],
    'num_credits': num_credits,
    'interest_rates': [0.07, 0.075]  # base rates: initial 7.0%, then 7.5%
}

# Usage:
credit_generator = CreditGenerator(**credit_generator_inputs, random_seed=42)
credits_df = credit_generator.generate_credits()
print('Top 5 rows of generated credits:')
display(credits_df.head())
print('\nCredits DataFrame summary statistics:')
display(credits_df.describe())
print('\nCredits DataFrame info:')
display(credits_df.info())


Top 5 rows of generated credits:


,Type,Principal,Maturity,Credit_Spread,Delta_FV
0,Cash Loan Credit,10860,6,0.012,-95.590
1,Cash Loan Credit,16265,6,0.006,-146.740
2,Cash Loan Credit,26023,6,0.005,-235.510
3,Car Loan Credit,22433,6,0.018,-193.330
4,Cash Loan Credit,38693,4,0.009,-227.380



Credits DataFrame summary statistics:


,Principal,Maturity,Credit_Spread,Delta_FV
count,"10,000.000","10,000.000","10,000.000","10,000.000"
mean,"31,473.846",5.510,0.011,-252.342
std,"13,317.703",1.242,0.005,122.939
min,"10,002.000",2.000,0.005,"-1,218.070"
25%,"20,841.250",4.000,0.008,-325.750
50%,"30,946.500",5.000,0.011,-236.520
75%,"41,113.500",7.000,0.013,-158.657
max,"99,912.000",11.000,0.040,-57.980



Credits DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Type           10000 non-null  object 
 1   Principal      10000 non-null  int64  
 2   Maturity       10000 non-null  int64  
 3   Credit_Spread  10000 non-null  float64
 4   Delta_FV       10000 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 390.8+ KB


None

In [6]:
swaps_df, credits_df_v1 = SwapGenerator(credits_df, num_swaps, fulfillment_ratio, random_factor=0.95)
print('Top 5 rows of generated swaps:')
display(swaps_df.head())
print('\nSwaps DataFrame summary statistics:')
display(swaps_df.describe())
print('\nSwaps DataFrame info:')
display(swaps_df.info())

Top 5 rows of generated swaps:


,Swap,Principal,Delta_FV,Maturity
0,1,"93,840,688.000","712,295.959",5.500
1,2,"95,030,488.000","723,897.444",5.530



Swaps DataFrame summary statistics:


,Swap,Principal,Delta_FV,Maturity
count,2.000,2.000,2.000,2.000
mean,1.500,"94,435,588.000","718,096.702",5.515
std,0.707,"841,315.648","8,203.489",0.021
min,1.000,"93,840,688.000","712,295.959",5.500
25%,1.250,"94,138,138.000","715,196.330",5.508
50%,1.500,"94,435,588.000","718,096.702",5.515
75%,1.750,"94,733,038.000","720,997.073",5.522
max,2.000,"95,030,488.000","723,897.444",5.530



Swaps DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Swap       2 non-null      int64  
 1   Principal  2 non-null      float64
 2   Delta_FV   2 non-null      float64
 3   Maturity   2 non-null      float64
dtypes: float64(3), int64(1)
memory usage: 192.0 bytes


None

## Cplex Model for First Instance

In [7]:
import time
start_time = time.time()

assignment, delta, status, mdl = solve_with_cplex(credits_df_v1, swaps_df,verbose=False, time_limit=60,
                                                  num_cpu=None, mip_gap=1e-2, rel_tol=1e-4,
                                                    abs_tol=1e-3,precision=2,
                                                    presolve=1, reduce=1,
                                                  experiment_name=f'Validation_Example_{num_credits}_Credits_{num_swaps}')



t_0_time = time.time() - start_time
deterministic_time = mdl.solve_details.deterministic_time
print(f"Wall Time: {t_0_time:.2f} seconds")
print(f"Deterministic Time: {deterministic_time:.2f} seconds")

integer optimal solution
Solution status: CPXMIP_OPTIMAL with delta = 0.0499875856379004
Wall Time: 2.32 seconds
Deterministic Time: 1537.74 seconds


In [8]:
delta

0.0499875856379004

## Model Validation and Metrics

In [9]:
results_df, summary = validate_solution_cplex(
    experiment_name=f'Validation_Example_{num_credits}_Credits_{num_swaps}',
    mdl=mdl,
    wall_time=t_0_time,
    assignment=assignment,
    swaps_df=swaps_df,
    credits_df=credits_df_v1,
    objective_delta=delta,
    solver_name='CPLEX',
)

In [10]:
results_df.T

,0,1
Swap_ID,0,1
Assigned_Principal,"93,843,723.000","95,044,379.000"
Swap_Principal,"93,840,688.000","95,030,488.000"
Principal_OK,True,True
Assigned_Delta_FV,"-747,761.730","-760,083.330"
Swap_Delta_FV,"712,295.960","723,897.440"
r_j,1.050,1.050
Delta_OK,True,True
Assigned_Weighted_Maturity,5.500,5.530
Swap_Maturity,5.500,5.530


In [11]:
pd.DataFrame.from_dict(summary,orient='index')

,0
Experiment_Name,Validation_Example_10000_Credits_2
Solver,CPLEX
Deterministic_Time,"1,537.741"
Wall_Time,2.317
MIP_Gap,0.000
Objective_Delta,0.0500
All_Delta_OK,True
All_Principal_OK,True
All_Maturity_OK,True


## CBC Model for First Instance

In [12]:
assignment, delta, status, mdl = solve_with_cbc(
    credits_df_v1,
    swaps_df,
    verbose=False,
    time_limit=60,
    num_cpu=None,
    mip_gap=1e-2,
    experiment_name=f'Validation_Example_{num_credits}_Credits_{num_swaps}'
)

In [17]:
delta

0.035473121

In [18]:
results_df, summary =validate_solution_cbc(
    experiment_name=f'Validation_Example_{num_credits}_Credits_{num_swaps}',
    wall_time=t_0_time,
    assignment=assignment,
    swaps_df=swaps_df,
    credits_df=credits_df_v1,
    objective_delta=delta,
    solver_name='CBC',
)

In [19]:
results_df.T

,0,1
Swap_ID,0,1
Assigned_Principal,"93,854,331.000","95,036,552.000"
Swap_Principal,"93,840,688.000","95,030,488.000"
Principal_OK,True,True
Assigned_Delta_FV,"-737,563.320","-749,561.310"
Swap_Delta_FV,"712,295.960","723,897.440"
r_j,1.035,1.035
Delta_OK,True,True
Assigned_Weighted_Maturity,5.500,5.530
Swap_Maturity,5.500,5.530


In [20]:
pd.DataFrame.from_dict(summary,orient='index')      

,0
Experiment_Name,Validation_Example_10000_Credits_2
Solver,CBC
Wall_Time,2.317
Objective_Delta,0.0355
All_Delta_OK,True
All_Principal_OK,True
All_Maturity_OK,True
